# Mise au point Simulateur prime activité

In [33]:
from openfisca_france.entities import entities
from openfisca_france import FranceTaxBenefitSystem
from openfisca_core.simulation_builder import SimulationBuilder

from openfisca_france.scenarios import init_single_entity

import numpy as np
import pandas as pd

import plotly.express as px

from pprint import pprint

In [34]:
legislation_france = FranceTaxBenefitSystem()

In [35]:
TEST_CASE = {
    'individus': {
        'parent1': {
            #'age': {'2024-01': 46},
            'date_naissance': {'ETERNITY': '1978-06-02'},
            #'salaire_de_base': {'2023': 23115,'2024': 23115, '2025': 23115}
            'salaire_net': {'2023': 23115,'2024': 23115, '2025': 23115},
            'revenus_capital':{'2021': 0,'2022': 0,'2023': 0,'2024': 0, '2025': 3600}
        },
        'enfant1': {
            'date_naissance': {'ETERNITY': '2010-07-28'}
#            'age': {'2024-01': 8}
        },
        'enfant2': {
            'date_naissance': {'ETERNITY': '2013-04-19'}
#            'age': {'2024-01': 12}
        }
    }
}
TEST_CASE['foyers_fiscaux'] = {
    'foyer_fiscal1': {
        'declarants': ['parent1'],
        'personnes_a_charge': ['enfant1', 'enfant2'],
        'celibataire_ou_divorce': {'2023': True, '2024': True, '2025': True},
        'caseT': {'2023': True, '2024': True, '2025': True}
    }
}

TEST_CASE['menages'] = {
    'menage1': {
        'personne_de_reference': ['parent1'],
        'enfants': ['enfant1', 'enfant2'],
        'statut_occupation_logement': {'2023': 'primo_accedant', '2024': 'primo_accedant'}
        #'statut_occupation_logement': {'2023': 'locataire_vide', '2024': 'locataire_vide', '2025': 'locataire_vide'}
    }
}

TEST_CASE['familles'] = {
    'famille1': {
        'parents': ['parent1'],
        'enfants': ['enfant1', 'enfant2'],
        'rsa_forfait_asf': {'2023': 0, '2024': 0},
        'af': {'2023': 0, '2024': 0},
    }
}


In [36]:
simulation_builder = SimulationBuilder()
simulation_PPA = simulation_builder.build_from_entities(legislation_france, TEST_CASE)

#print(simulation_PPA.calculate('ppa', '2023-09'))
#print(simulation_PPA.calculate('ppa', '2025-09'))
def elements_calculs_ppa(simulation_PPA, period, array=False):
    forfait_logement = simulation_PPA.calculate('ppa_forfait_logement', period)
    elig = simulation_PPA.calculate('ppa_eligibilite', period)
    montant_forfaitaire_familialise = simulation_PPA.calculate('ppa_fictive_montant_forfaitaire', period)
    ppa_base_ressources = simulation_PPA.calculate('ppa_base_ressources', period)
    ppa_fictive_ressource_activite = simulation_PPA.calculate('ppa_fictive_ressource_activite', period)
    bonification_i = simulation_PPA.calculate('ppa_bonification', period)
    ppa_fictive = simulation_PPA.calculate('ppa_fictive', period)
    ppa = simulation_PPA.calculate('ppa', period)
    #bonification = simulation_PPA.calculate(bonification_i)
    if array == False : 
        return {'elig': elig[0],
                'ppa': ppa[0],
                'forfait_logement': forfait_logement[0],  
                'montant_forfaitaire_familialise': montant_forfaitaire_familialise[0], 
                'ppa_base_ressources': ppa_base_ressources[0], 
                'ppa_fictive_ressource_activite': ppa_fictive_ressource_activite[0], 
                'bonification_i': bonification_i[0],
                'ppa_fictive': ppa_fictive[0]}
    else:
        return {'elig': elig,
                'ppa': ppa,
                'forfait_logement': forfait_logement,  
                'montant_forfaitaire_familialise': montant_forfaitaire_familialise, 
                'ppa_base_ressources': ppa_base_ressources, 
                'ppa_fictive_ressource_activite': ppa_fictive_ressource_activite, 
                'bonification_i': bonification_i,
                'ppa_fictive': ppa_fictive}

period_sim = '2024-05'
elem_ppa = elements_calculs_ppa(simulation_PPA, period_sim)
ppa_montant_base = (
            elem_ppa['montant_forfaitaire_familialise']
            + elem_ppa['bonification_i']
            + elem_ppa['ppa_fictive_ressource_activite']
            - elem_ppa['ppa_base_ressources']
            - elem_ppa['forfait_logement']
            )
ppa_deduction = (
             elem_ppa['montant_forfaitaire_familialise']
            -  elem_ppa['ppa_base_ressources']
            -  elem_ppa['forfait_logement']
            )
ppa_fictive = ppa_montant_base - max(ppa_deduction, 0)
ppa_fictive = max(ppa_fictive, 0)

if(False):
    prestations = [
            'paje_base',
            'paje_clca',
            'paje_prepare',
            'paje_colca',
            'rsa_forfait_asf'
            ]
    for prest in prestations:
        print(prest, simulation_PPA.calculate(prest, period_sim))

print("Prime d'activité:", elem_ppa['ppa'])
print('Prestations familiales prises en compte: ', simulation_PPA.calculate('ppa_base_ressources_prestations_familiales', period_sim))
print("Allocation familiales", simulation_PPA.calculate('af', period_sim))
print(simulation_PPA.calculate('salaire_net', period_sim))
print(simulation_PPA.calculate('ppa_ressources_hors_activite_individu', period_sim))


Prime d'activité: 333.01733
Prestations familiales prises en compte:  [0.]
Allocation familiales [0.]
[1926.25    0.      0.  ]
[0. 0. 0.]


In [37]:


print("Montant forfaitaire:", simulation_PPA.calculate('ppa_fictive_montant_forfaitaire', period_sim)[0])
print("Montant de la bonification: ", simulation_PPA.calculate('ppa_bonification', period_sim)[0])
print("Montant base ressource: ", simulation_PPA.calculate('ppa_base_ressources', period_sim)[0])
print("Montant fictive: ", simulation_PPA.calculate('ppa_fictive_ressource_activite', period_sim)[0])
print("Montant forfait logement: ", simulation_PPA.calculate('ppa_forfait_logement', period_sim)[0])

print("Montant de la PPA: ", simulation_PPA.calculate('ppa', period_sim)[0])


Montant forfaitaire: 1120.734
Montant de la bonification:  181.19
Montant base ressource:  1926.25
Montant fictive:  1175.0126
Montant forfait logement:  184.92111
Montant de la PPA:  333.01733


## Simulation évolution du salaire

In [38]:
TEST_CASE_SIMU = TEST_CASE.copy()
TEST_CASE_SIMU['axes']= [[{'count':200, 'name':'salaire_net', 'min':0, 'max': 50000, 'period':'2024'}]]
simulation_builder = SimulationBuilder()
simulation_per = simulation_builder.build_from_entities(legislation_france, TEST_CASE_SIMU)
sdb = simulation_per.calculate_add('salaire_net', '2024')
salaire = sdb.reshape(200,3).sum(1)
ppa = simulation_per.calculate('ppa', '2024-05')

pd.DataFrame.from_dict({'salaire': salaire, 'ppa': ppa})


,salaire,ppa
0,0.000000,0.000000
1,251.256271,0.000000
2,502.512543,25.544449
3,753.768982,38.316608
4,1005.025085,51.088764
...,...,...
195,48994.972656,0.000000
196,49246.222656,0.000000
197,49497.480469,0.000000
198,49748.734375,0.000000


In [39]:
resultats_simu_ppa = elements_calculs_ppa(simulation_per, '2024-05', array=True)
resultats_simu_ppa['bonification_i'] = resultats_simu_ppa['bonification_i'].reshape(200, 3).sum(1)
resultats_simu_ppa['salaire'] = salaire
resultats_simu_ppa['salaire mensuel'] = salaire / 12
df_resultats_simu_ppa = pd.DataFrame.from_dict(resultats_simu_ppa)
df_resultats_simu_ppa

,elig,ppa,forfait_logement,montant_forfaitaire_familialise,ppa_base_ressources,ppa_fictive_ressource_activite,bonification_i,ppa_fictive,salaire,salaire mensuel
0,True,0.000000,184.921112,1120.734009,0.000000,0.000000,0.000000,0.000031,0.000000,0.000000
1,True,0.000000,184.921112,1120.734009,20.938025,12.772195,0.000000,12.772189,251.256271,20.938023
2,True,25.544449,184.921112,1120.734009,41.876049,25.544390,0.000000,25.544470,502.512543,41.876045
3,True,38.316608,184.921112,1120.734009,62.814072,38.316586,0.000000,38.316631,753.768982,62.814083
4,True,51.088764,184.921112,1120.734009,83.752098,51.088779,0.000000,51.088787,1005.025085,83.752090
...,...,...,...,...,...,...,...,...,...,...
195,True,0.000000,184.921112,1120.734009,4082.914795,2490.578125,181.190002,0.000000,48994.972656,4082.914307
196,True,0.000000,184.921112,1120.734009,4103.852539,2503.350098,181.190002,0.000000,49246.222656,4103.852051
197,True,0.000000,184.921112,1120.734009,4124.790527,2516.122314,181.190002,0.000000,49497.480469,4124.790039
198,True,0.000000,184.921112,1120.734009,4145.728516,2528.894531,181.190002,0.000000,49748.734375,4145.728027


In [42]:
px.line(df_resultats_simu_ppa, x='salaire mensuel', y='ppa')

Effet de l'augmentation de revenus sur la prime d'activité : on perd 39% de l'augmentation.

In [48]:
df_red = df_resultats_simu_ppa[df_resultats_simu_ppa['salaire mensuel'] < 2000]
df_red =df_red[df_red['salaire mensuel'] > 1500]
np.gradient(df_red['ppa'], df_red['salaire mensuel'])

array([-0.39000082, -0.38999653, -0.3900013 , -0.39000225, -0.38999844,
       -0.3900013 , -0.39000225, -0.3900013 , -0.38999367, -0.3899994 ,
       -0.39000702, -0.38999748, -0.38999367, -0.3899994 , -0.39000416,
       -0.38999748, -0.38999653, -0.3900032 , -0.3900032 , -0.38999796,
       -0.3899989 , -0.39000368, -0.38999557, -0.3899901 ], dtype=float32)